In [60]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from pathlib import Path
from fastai.datasets import Config
import pandas as pd
import numpy as np
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook, tqdm
from efficientnet_pytorch import EfficientNet
from torch_lr_finder import LRFinder

import pickle
from tqdm import tqdm_notebook

import torch
import torchvision
import gc
import torch.nn as nn
import torch.optim as optim
import time
import os
import copy

# base_path = Config.data_path()
base_path = Path('/mnt/e/MojePliki/Programy/kaggle_data')
data_path = base_path/'Steel_Defects_Detection'
data_path.mkdir(parents=True, exist_ok=True)

with open(data_path/'defect_labels.pickle', 'rb') as f:
    defects_labels = pickle.load(f)
    
DEVICE = torch.device('cuda' if torch.cuda.is_available() else "cpu")


/home/krzysiek/anaconda3/envs/kaggle_sandbox/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
#!pip install --upgrade efficientnet-pytorch
#!pip install torch-lr-finder

In [52]:
class SteelDefects_Train_Dataset(Dataset):
    def __init__(self, transforms, images_path=data_path/'train_images', labels_dict=defects_labels):
        super().__init__()
        self.images = sorted(images_path.glob('*.jpg'))
        self.labels_dict = labels_dict
        self.transforms = transforms
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
#         label = torch.tensor(self.labels[idx])
        label = self.labels_dict[image_path.name]
        
        raw_image = Image.open(image_path)
        transformed_image = self.transforms(raw_image)
        
        return transformed_image, label


class SteelDefects_Test_Dataset(Dataset):
    def __init__(self, transforms, images_path=data_path/'test_images'):
        super().__init__()
        self.images = sorted(images_path.glob('*.jpg'))
        self.transforms = transforms
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]

        raw_image = Image.open(image_path)
        transformed_image = self.transforms(raw_image)
        
        return transformed_image

    
def get_train_valid_loader(batch_size=8,
                           random_seed=2019,
                           valid_size=0.1,
                           shuffle=True,
                           show_sample=False,
                           pin_memory=True):
    # From: https://gist.github.com/kevinzakka/d33bf8d6c7f06a9d8c76d97a7879f5cb
    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg

    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    valid_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
    ])
    
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])

    # load the dataset
    train_dataset = SteelDefects_Train_Dataset(transforms=train_transform)

    valid_dataset = SteelDefects_Train_Dataset(transforms=valid_transform)

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size, 
        sampler=train_sampler,
        pin_memory=True
    )
    
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, 
        batch_size=batch_size, 
        sampler=valid_sampler, 
        pin_memory=True,
    )

    # visualize some images
    if show_sample:
        sample_loader = torch.utils.data.DataLoader(
            valid_dataset, batch_size=9, 
            shuffle=shuffle, pin_memory=pin_memory,
        )
        data_iter = iter(sample_loader)
        images, labels = data_iter.next()
        X = images.numpy().transpose([0, 2, 3, 1])
        plot_images(X, labels)

    dataloaders = {
        'train': train_loader, 
        'valid': valid_loader
    }
    
    data_sizes = {
        'train': len(train_dataset), 
        'valid': len(valid_dataset)
    }
    
    return dataloaders, data_sizes


def get_test_loader(batch_size=8):
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

    # define transform
    transform = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])

    dataset = SteelDefects_Test_Dataset(transforms=transform)

    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=False, pin_memory=True
    )

    return data_loader


label_names = [
    'No defect',
    'Defect'
]


def plot_images(images, cls_true, cls_pred=None):
    """
    Adapted from https://github.com/Hvass-Labs/TensorFlow-Tutorials/
    """
    fig, axes = plt.subplots(3, 3)

    for i, ax in enumerate(axes.flat):
        # plot img
        ax.imshow(images[i, :, :, :], interpolation='spline16')

        # show true & predicted classes
        cls_true_name = label_names[cls_true[i]]
        if cls_pred is None:
            xlabel = "{0} ({1})".format(cls_true_name, cls_true[i])
        else:
            cls_pred_name = label_names[cls_pred[i]]
            xlabel = "True: {0}\nPred: {1}".format(
                cls_true_name, cls_pred_name
            )
        ax.set_xlabel(xlabel)
        ax.set_xticks([])
        ax.set_yticks([])

    plt.show()

In [53]:
dataloaders, data_sizes = get_train_valid_loader()

In [51]:
# mean = 0.
# std = 0.
# loader = dataloaders['valid']
# nb_samples = len(loader)
# for data, labels in tqdm(loader):
#     print(labels)
#     batch_samples = data.size(0)
#     data = data.view(batch_samples, data.size(1), -1)
#     mean += data.mean(2).sum(0)
#     std += data.std(2).sum(0)
#     break
# mean /= nb_samples
# std /= nb_samples


  0%|          | 0/157 [00:00<?, ?it/s]

tensor([0, 0, 1, 1, 0, 0, 0, 1])


In [3]:
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'valid': transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }

## Transfer learning

In [57]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.effnet =  EfficientNet.from_pretrained('efficientnet-b0')
        self.l1 = nn.Linear(1000 , 256)
        self.dropout = nn.Dropout(0.5)
        self.l2 = nn.Linear(256, 2) # Binary clasifier for now
        self.relu = nn.LeakyReLU()
        
    def forward(self, input):
        x = self.effnet(input)
        x = x.view(x.size(0),-1)
        x = self.dropout(self.relu(self.l1(x)))
        x = self.l2(x)
        
        return x
    
classifier = Classifier().to(DEVICE)

Downloading: "http://storage.googleapis.com/public-models/efficientnet/efficientnet-b0-355c32eb.pth" to /home/krzysiek/.torch/models/efficientnet-b0-355c32eb.pth

0it [00:00, ?it/s]
122880it [00:00, 857636.96it/s]
212992it [00:00, 861372.70it/s]
311296it [00:00, 887831.11it/s]
466944it [00:00, 946884.66it/s]
589824it [00:00, 1014103.67it/s]
688128it [00:00, 863595.00it/s] 
802816it [00:00, 907268.77it/s]
892928it [00:00, 887868.40it/s]
1015808it [00:01, 957581.40it/s]
1114112it [00:01, 881842.42it/s]
1204224it [00:01, 885572.98it/s]
1327104it [00:01, 949548.32it/s]
1425408it [00:01, 873002.53it/s]
1515520it [00:01, 868227.01it/s]
1613824it [00:01, 880564.62it/s]
1736704it [00:01, 958457.77it/s]
1843200it [00:02, 852101.82it/s]
1949696it [00:02, 863225.98it/s]
2048000it [00:02, 895001.92it/s]
2170880it [00:02, 969769.75it/s]
2277376it [00:02, 895727.47it/s]
2375680it [00:02, 918350.76it/s]
2473984it [00:02, 930894.49it/s]
2572288it [00:02, 936751.62it/s]
2670592it [00:02, 880102.33it/s]

Loaded pretrained weights for efficientnet-b0


In [64]:
optimizer_ft = optim.Adam(classifier.parameters(), lr=0.0000001)
criterion = nn.CrossEntropyLoss()
lr_finder = LRFinder(classifier, optimizer_ft, criterion, device=DEVICE)
lr_finder.range_test(dataloaders['train'], end_lr=1, num_iter=500)
lr_finder.reset()
lr_finder.plot()

ImportError: IntProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
def train_model(model,criterion,num_epochs=10,lr=0.0003,batch_size=8):
    since = time.time()
    model.to(device)
    best_acc = 0.0
    i = 0
    losses = []
    accuracy = []
    earlystop = EarlyStopping(patience = 2,verbose = True)
    for epoch in range(num_epochs):
        print('Epoch:',epoch)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        lr = lr*0.8
        if(epoch%10==0):
            lr = 0.0001
        
        for phase in ['train','val']:
            if phase == ' train':
                model.train()  
            else:
                model.eval()       
            running_loss = 0.0
            running_corrects = 0
            total = 0 
            j = 0
            for  batch_idx, (data, target) in enumerate(dataloaders[phase]):    
                data, target = Variable(data), Variable(target)
                data = data.type(torch.cuda.FloatTensor)
                target = target.type(torch.cuda.LongTensor)
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                _, preds = torch.max(output, 1)
                running_corrects = running_corrects + torch.sum(preds == target.data)
                running_loss += loss.item() * data.size(0)
                j = j+1
                if(phase =='train'):
                    loss.backward()
                    optimizer.step()

                if batch_idx % 300 == 0:
                    print('{} Epoch: {}  [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tAcc: {:.6f}'.format(phase,epoch, batch_idx * len(data), len(dataloaders[phase].dataset),100. * batch_idx / len(dataloaders[phase])
                                                                                                 , running_loss/(j*batch_size),running_corrects.double()/(j*batch_size)))
            epoch_acc = running_corrects.double()/(len(dataloaders[phase])*batch_size)
            epoch_loss = running_loss/(len(dataloaders[phase])*batch_size)
            if(phase == 'val'):
                earlystop(epoch_acc,model)
            
            if(phase == 'Train'):
                losses.append(epoch_loss)
                accuracy.append(epoch_acc)
        if(earlystop.early_stop):
            print("Early stopping")
            break
        print('{} Accuracy: '.format(phase),epoch_acc.item())
    return losses,accuracy